# Play ROI and videos from two separate sessions

In [1]:
import bokeh.plotting as bpl
import cv2
import glob
import holoviews as hv
from IPython import get_ipython
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import psutil
import pandas as pd
import tifffile

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.source_extraction.cnmf.cnmf import load_CNMF
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.utils.visualization import view_quilt
from caiman.utils.visualization import get_contours
from caiman.motion_correction import high_pass_filter_space
from caiman.summary_images import correlation_pnr

from ipywidgets import IntSlider, VBox
import fastplotlib as fpl
from fastplotlib.widgets import ImageWidget

# load local package/
import sys
root_dir = os.path.split(os.getcwd())[0] # get root
utils_dir = os.path.join(root_dir,'utils') # get utils folder path
sys.path.append(utils_dir) # add it to system path (not ideal) - doing this to reduce pip installs for local lab usage
import plot_tools as fpl_tools # import movie

def autocluster():
    #%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
    print(f"You have {psutil.cpu_count()} CPUs available in your current environment")
    num_processors_to_use = None
    print("Setting up new cluster")
    _, cluster, n_processes = cm.cluster.setup_cluster(backend='multiprocessing', 
                                                     n_processes=num_processors_to_use, 
                                                     ignore_preexisting=False)
    print(f"Successfully set up cluster with {n_processes} processes")
    return cluster, n_processes

caiman package loaded...
Suite2p not detected...


`hdf5_fnames` refers to the paths to .hdf5 outputs from CaImAn

In [2]:
hdf5_fnames = [r'/Users/js0403/miniscope/PFC-Neurons/122D/AAV2/3-Syn-GCaMP8f/2024_02_06/12_21_26/miniscopeDeviceName/full_movie_20240320222511_cnmfe.hdf5',
               r'/Users/js0403/miniscope/PFC-Neurons/122D/AAV2/3-Syn-GCaMP8f/2024_03_05/12_51_33/miniscopeDeviceName/full_movie_20240320224747_cnmfe.hdf5',
              r'/Users/js0403/miniscope/PFC-Neurons/122D/AAV2/3-Syn-GCaMP8f/2024_03_15/10_57_51/miniscopeDeviceName/full_movie_20240320185923_cnmfe.hdf5']

# start cluster
if 'cluster' in locals():
    cluster.terminate()
cluster, n_processes = autocluster()

# load data
images_list = []; cm_list = []
for i in hdf5_fnames:
    # move directories
    root_path = os.path.split(i)[0] # extract root
    mov_path = os.path.join(root_path,'full_movie.tif')
    print("Working in:",root_path)

    # load caiman objects
    cnmfe_model = load_CNMF(i)

    # load memmory mapped file
    print("Loading:",cnmfe_model.mmap_file)
    Yr, dims, num_frames = cm.load_memmap(cnmfe_model.mmap_file)
    images = np.reshape(Yr.T, [num_frames] + list(dims), order='F')
    images_list.append(images) # cache

    # Threshold data for CNMFE
    min_SNR = 3      # SNR threshold - this could be lowered as needed, but you will get noise
    rval_thr = 0.99  # spatial correlation threshold - You should generally keep this
    
    # extract params
    cnmfe_params = cnmfe_model.params.__dict__
    
    # We need to reassign the fnames variable
    try: cnmfe_model.params.data['fnames'] = [cnmfe_model.params.data['fnames'][0].decode('UTF-8')]; 
    except: pass # dont change
    fr = cnmfe_model.params.data['fr'] # frame rate
    
    # changing parameters
    quality_params = {'min_SNR': min_SNR,
                      'rval_thr': rval_thr,
                      'use_cnn': False}
    cnmfe_model.params.change_params(params_dict=quality_params)
    
    # This is where the parameters above get put to test
    cnmfe_model.estimates.evaluate_components(images, cnmfe_model.params, dview=cluster)
    print('*****')
    print(f"Total number of components: {len(cnmfe_model.estimates.C)}")
    print(f"Number accepted: {len(cnmfe_model.estimates.idx_components)}")
    print(f"Number rejected: {len(cnmfe_model.estimates.idx_components_bad)}")

    # store caiman cnmf object
    cm_list.append(cnmfe_model)

You have 12 CPUs available in your current environment
Setting up new cluster
Successfully set up cluster with 11 processes
Working in: /Users/js0403/miniscope/PFC-Neurons/122D/AAV2/3-Syn-GCaMP8f/2024_02_06/12_21_26/miniscopeDeviceName
Loading: /Users/js0403/miniscope/PFC-Neurons/122D/AAV2/3-Syn-GCaMP8f/2024_02_06/12_21_26/miniscopeDeviceName/motioncorr_d1_600_d2_600_d3_1_order_C_frames_2851.mmap
*****
Total number of components: 15
Number accepted: 11
Number rejected: 4
Working in: /Users/js0403/miniscope/PFC-Neurons/122D/AAV2/3-Syn-GCaMP8f/2024_03_05/12_51_33/miniscopeDeviceName
Loading: /Users/js0403/miniscope/PFC-Neurons/122D/AAV2/3-Syn-GCaMP8f/2024_03_05/12_51_33/miniscopeDeviceName/motioncorr_d1_600_d2_600_d3_1_order_C_frames_3000.mmap
*****
Total number of components: 35
Number accepted: 20
Number rejected: 15
Working in: /Users/js0403/miniscope/PFC-Neurons/122D/AAV2/3-Syn-GCaMP8f/2024_03_15/10_57_51/miniscopeDeviceName
Loading: /Users/js0403/miniscope/PFC-Neurons/122D/AAV2/3-Sy

In [12]:
# instantiate object
cnmf_plotter = fpl_tools.play_caiman_movie(images=images_list[0],cnmf_object=cm_list[0])
mov = cnmf_plotter.play_movie_draw_roi(components_type='accepted',show_movie=False)
mov.show(sidecar=True)

RFBOutputContext()

/Users/js0403/anaconda3/envs/caiman/lib/python3.11/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='auto-scale scene'), Button(icon='align-center', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='auto-center scene'), ToggleButton(value=True, icon='hand-pointer', layout=Layout(width='auto'), tooltip='panzoom controller'), ToggleButton(value=True, description='1:1', layout=Layout(width='auto'), style=ToggleButtonStyle(font_weight='bold'), tooltip='maintain aspect'), Button(icon='arrow-down', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='y-axis direction'), Button(icon='draw-polygon', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='add PolygonSelector'), ToggleButton(value=False, icon='video', layout=Layout(width='auto'), tooltip='record'), Dropdown(description='Subplots:', layout=Layout(width='200px'), options=('(0, 0)',), value='(0, 0)'))), IpywidgetImageWidgetToolbar(children=(HBox

In [13]:
cnmf_plotter = fpl_tools.play_caiman_movie(images=images_list[1],cnmf_object=cm_list[1])
mov1 = cnmf_plotter.play_movie_draw_roi(components_type='accepted',show_movie=False)
mov1.show(sidecar=True)

RFBOutputContext()

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='auto-scale scene'), Button(icon='align-center', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='auto-center scene'), ToggleButton(value=True, icon='hand-pointer', layout=Layout(width='auto'), tooltip='panzoom controller'), ToggleButton(value=True, description='1:1', layout=Layout(width='auto'), style=ToggleButtonStyle(font_weight='bold'), tooltip='maintain aspect'), Button(icon='arrow-down', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='y-axis direction'), Button(icon='draw-polygon', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='add PolygonSelector'), ToggleButton(value=False, icon='video', layout=Layout(width='auto'), tooltip='record'), Dropdown(description='Subplots:', layout=Layout(width='200px'), options=('(0, 0)',), value='(0, 0)'))), IpywidgetImageWidgetToolbar(children=(HBox

In [14]:
cnmf_plotter = fpl_tools.play_caiman_movie(images=images_list[2],cnmf_object=cm_list[2])
mov1 = cnmf_plotter.play_movie_draw_roi(components_type='accepted',show_movie=False)
mov1.show(sidecar=True)

RFBOutputContext()

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='auto-scale scene'), Button(icon='align-center', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='auto-center scene'), ToggleButton(value=True, icon='hand-pointer', layout=Layout(width='auto'), tooltip='panzoom controller'), ToggleButton(value=True, description='1:1', layout=Layout(width='auto'), style=ToggleButtonStyle(font_weight='bold'), tooltip='maintain aspect'), Button(icon='arrow-down', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='y-axis direction'), Button(icon='draw-polygon', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='add PolygonSelector'), ToggleButton(value=False, icon='video', layout=Layout(width='auto'), tooltip='record'), Dropdown(description='Subplots:', layout=Layout(width='200px'), options=('(0, 0)',), value='(0, 0)'))), IpywidgetImageWidgetToolbar(children=(HBox

# Now lets do some plotting